In [12]:
"""
This project allows user to upload to input files:
It uploads an existing resume file and an job description, both in markdown format. 
The chatbot use LangChain Prompt Template to generate prommpt, and provide two pieces of feedback.
provide relevant suggestion feedback. Specficialy, 

 **Action-Driven Results**:  
   - Use **strong action verbs** and **quantifiable results** (e.g., \
percentages, revenue, efficiency improvements) to highlight impact.  

 **Keyword Optimization**:  
   - Integrate **keywords** and phrases from the job description naturally to \
optimize for ATS (Applicant Tracking Systems). 

2. **Additional Suggestions (if applicable)**:  
   - List any skills that could improve alignment with the role.
   - Recommend certifications or courses that could be pursued.
   - Suggest specific projects or experiences to develop further.


"""

'\nThis project allows user to upload to input files:\nIt uploads an existing resume file and an job description, both in markdown format. \nThe chatbot use LangChain Prompt Template to generate prommpt, and provide two pieces of feedback.\nprovide relevant suggestion feedback. Specficialy, \n\n **Action-Driven Results**:  \n   - Use **strong action verbs** and **quantifiable results** (e.g., percentages, revenue, efficiency improvements) to highlight impact.  \n\n **Keyword Optimization**:  \n   - Integrate **keywords** and phrases from the job description naturally to optimize for ATS (Applicant Tracking Systems). \n\n2. **Additional Suggestions (if applicable)**:  \n   - List any skills that could improve alignment with the role.\n   - Recommend certifications or courses that could be pursued.\n   - Suggest specific projects or experiences to develop further.\n\n\n'

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
from sk import my_sk
from markdown import markdown
from openai import OpenAI

/Users/zhouyu/Documents/notebook/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# define chatbot
chat_model = ChatOpenAI(openai_api_key=my_sk, temperature=0)

with open("resume.md", "r", encoding="utf-8") as file:
    resume_string = file.read()

with open("jd.md", "r", encoding="utf-8") as file:
    job_description_string = file.read()
    jd_string = file.read()


/var/folders/ww/xnbg6qmx0q9gls3523dt2lg80000gn/T/ipykernel_94100/883962386.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(openai_api_key=my_sk, temperature=0)


In [26]:
# define prompt template
prompt_template_resume = """
**Objective:**  
Please review my resume and optimize it for the specific job described in the provided job description. 

**Input Files:**  
1. **My Resume**: {resume}
2. **Job Description**: {job_description}

### Guidelines:

1. **Relevance**:  
   - Focus on experiences, skills, and achievements that are most relevant to the job description.
   - Remove or downplay irrelevant details to create a concise and targeted resume.
   - Limit the work experience section to 2-3 of the most relevant roles.
   - Restrict bullet points under each role to 2-3 key impacts.

2. **Action-Driven Results**:  
   - Use strong action verbs and include quantifiable results (e.g., percentages, revenue increases) to demonstrate impact.

3. **Keyword Optimization**:  
   - Integrate keywords and phrases from the job description naturally to enhance optimization for Applicant Tracking Systems (ATS).

### Output:

1. **Tailored Resume**:  
   - Provide a revised resume in Markdown format that emphasizes relevant experiences, skills, and achievements.
   - Ensure it incorporates job description keywords for ATS optimization.
   - Maintain strong language throughout and keep it concise (ideally one page).

2. **Additional Suggestions (if applicable)**:  
   - List any skills that could improve alignment with the role.
   - Recommend certifications or courses that could be pursued.
   - Suggest specific projects or experiences to develop further.

"""

prompt_resume = PromptTemplate(input_variables=["resume", "job_description"], template = prompt_template_resume)

chain_job = LLMChain(
    llm=chat_model,
    prompt=prompt_resume,
)

response_lang_chain = chain_job.run({'resume':resume_string, 'job_description':job_description_string})

"# Zhou Yu\n\nEmail: yuuniverse4444@gmail.com  \nPhone: 650-704-6941\n\n## Work Experience\n\n### Data Science Consultant, Bond Capital (Mary Meeker's Growth fund)\n*2022.02 - 2024.9*\n\n- Led the development of data-driven investing roadmaps, implementing machine learning algorithms to improve recall by 14.3%.\n- Utilized SHAP values for feature importance and developed backtesting infrastructure using Spark/EMR.\n- Applied LLM to analyze web & product review data, improving output consistency by 10.2%.\n\n### Head of Data Science, Grow Therapy (Sequoia invested)\n*2022.2 – 2022.11*\n\n- Led strategic data science initiatives, resulting in a 10% conversion increase and 7% retention improvement.\n- Implemented provider ranking algorithms using mix-effect & ML models for optimization.\n- Created reporting dashboards to optimize revenue cycle management, increasing efficiency by 15%.\n\n## Technical Skills\n\n- Programming languages: Python, R, SQL, Spark/Scala\n- Tools: AWS (EMR, EC2, S

In [5]:
prompt_template = lambda resume_string, jd_string : f"""
You are a professional resume optimization expert specializing in tailoring \
resumes to specific job descriptions. Your goal is to optimize my resume and \
provide actionable suggestions for improvement to align with the target role.

### Guidelines:
1. **Relevance**:  
   - Prioritize experiences, skills, and achievements **most relevant to the \
job description**.  
   - Remove or de-emphasize irrelevant details to ensure a **concise** and \
**targeted** resume.
   - Limit work experience section to 2-3 most relevant roles
   - Limit bullet points under each role to 2-3 most relevant impacts

2. **Action-Driven Results**:  
   - Use **strong action verbs** and **quantifiable results** (e.g., \
percentages, revenue, efficiency improvements) to highlight impact.  

3. **Keyword Optimization**:  
   - Integrate **keywords** and phrases from the job description naturally to \
optimize for ATS (Applicant Tracking Systems).  

4. **Additional Suggestions** *(If Gaps Exist)*:  
   - If the resume does not fully align with the job description, suggest:  
     1. **Additional technical or soft skills** that I could add to make my \
profile stronger.  
     2. **Certifications or courses** I could pursue to bridge the gap.  
     3. **Project ideas or experiences** that would better align with the role.  

5. **Formatting**:  
   - Output the tailored resume in **clean Markdown format**.  
   - Include an **"Additional Suggestions"** section at the end with \
actionable improvement recommendations.  

---

### Input:
- **My resume**:  
{resume_string}

- **The job description**:  
{jd_string}

---

### Output:  
1. **Tailored Resume**:  
   - A resume in **Markdown format** that emphasizes relevant experience, \
skills, and achievements.  
   - Incorporates job description **keywords** to optimize for ATS.  
   - Uses strong language and is no longer than **one page**.

2. **Additional Suggestions** *(if applicable)*:  
   - List **skills** that could strengthen alignment with the role.  
   - Recommend **certifications or courses** to pursue.  
   - Suggest **specific projects or experiences** to develop.
"""

In [6]:
# create prompt
prompt = prompt_template(resume_string, jd_string)

# setup api client
client = OpenAI(api_key=my_sk)

# make api call
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Expert resume writer"},
        {"role": "user", "content": prompt}
    ], 
    temperature = 0
)

# extract response
response_string = response.choices[0].message.content

In [7]:
response_list = response_string.split("## Additional Suggestions")

In [8]:
response_list[1]

'\n1. **Skills to Add**:\n   - Familiarity with cloud platforms (e.g., Azure, GCP) if relevant to the job.\n   - Experience with specific data visualization tools (e.g., Tableau, Power BI) if applicable.\n\n2. **Certifications or Courses**:\n   - Consider pursuing certifications in advanced machine learning or data engineering (e.g., AWS Certified Machine Learning, Google Data Analytics).\n\n3. **Project Ideas**:\n   - Develop a project that showcases the application of LLMs in a specific industry (e.g., healthcare, finance) to demonstrate practical experience.\n   - Create a case study on optimizing a data pipeline for real-time analytics, highlighting your skills in data integration and processing.'

In [10]:
# save as PDF
output_pdf_file = "resume_new.pdf"

# Convert Markdown to HTML
html_content = markdown(response_list[0])
html_content

# # Convert HTML to PDF and save
# HTML(string=html_content).write_pdf(output_pdf_file, 
#                                     stylesheets=['resumes/style.css'])

"<h1>Zhou Yu</h1>\n<p>Email: yuuniverse4444@gmail.com<br />\nPhone: 650-704-6941</p>\n<h2>Work Experience</h2>\n<h3>Data Science Consultant, Bond Capital (Mary Meeker's Growth Fund)</h3>\n<p><em>2022.02 - 2024.09</em><br />\n- Developed a data-driven investing roadmap, implementing end-to-end machine learning algorithms that improved investment opportunity sourcing recall from 23.2% to 37.5% using xgBoost models.\n- Created a backtesting infrastructure with Spark/EMR, enhancing model performance evaluation and feature impact analysis.\n- Led the application of LLM to analyze web and product review data, increasing output format consistency from 89.4% to 99.6%.</p>\n<h3>Head of Data Science, Grow Therapy (Sequoia Invested)</h3>\n<p><em>2022.02 - 2022.11</em><br />\n- Spearheaded data science initiatives, resulting in a 10% increase in user conversion and a 7% boost in retention through optimized provider ranking algorithms.\n- Developed a reporting dashboard for medical claims, improvin

In [11]:
html_content_2 = markdown(response_list[1])
html_content_2

'<ol>\n<li><strong>Skills to Add</strong>:</li>\n<li>Familiarity with cloud platforms (e.g., Azure, GCP) if relevant to the job.</li>\n<li>\n<p>Experience with specific data visualization tools (e.g., Tableau, Power BI) if applicable.</p>\n</li>\n<li>\n<p><strong>Certifications or Courses</strong>:</p>\n</li>\n<li>\n<p>Consider pursuing certifications in advanced machine learning or data engineering (e.g., AWS Certified Machine Learning, Google Data Analytics).</p>\n</li>\n<li>\n<p><strong>Project Ideas</strong>:</p>\n</li>\n<li>Develop a project that showcases the application of LLMs in a specific industry (e.g., healthcare, finance) to demonstrate practical experience.</li>\n<li>Create a case study on optimizing a data pipeline for real-time analytics, highlighting your skills in data integration and processing.</li>\n</ol>'